In [ ]:
'''
Implement the image stitching for a 360 degree panoramic output. 
This should function in real-time. You can use any type of features.
 You can use built-in libraries/tools provided by OpenCV or DepthAI API. 
You cannot use any built-in function that does output = image_stitch(image1, image2). 
You are supposed to implement the image_stitch() function

'''

In [2]:
import cv2
import depthai as dai
import numpy as np

In [1]:
# General definition for stitching two images
def stitch_2_imgs(img1, img2, feature):
    # Convert images to grayscale
    img1_bw = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_bw = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Initialize feature detector based on user choice
    if feature == "SIFT":
        feat = cv2.SIFT_create() 
    elif feature == "ORB":
        feat = cv2.ORB_create()
    else:
        print("Please enter correct feature value.")
        return 
    
    # Detect and compute keypoints and descriptors for both images
    kp_img1, desc_img1 = feat.detectAndCompute(img1_bw, None) 
    kp_img2, desc_img2 = feat.detectAndCompute(img2_bw, None) 

    # Use Brute Force matcher to find matches between descriptors
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc_img2, desc_img1, k=2)

    # Filter good matches based on distance ratio
    good_points = []
    for m, n in matches: 
        if m.distance < 0.6 * n.distance: 
            good_points.append(m) 

    # Extract corresponding points from good matches
    query_pts = np.float32([kp_img2[m.queryIdx].pt for m in good_points]).reshape(-1, 1, 2) 
    train_pts = np.float32([kp_img1[m.trainIdx].pt for m in good_points]).reshape(-1, 1, 2) 

    # Find homography matrix using RANSAC algorithm
    matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 

    # Warp the second image onto the first image based on homography
    dst = cv2.warpPerspective(img2, matrix, ((img1.shape[1] + img2.shape[1]), img2.shape[0])) 

    # Paste the first image onto the stitched image
    dst[0:img1.shape[0], 0:img1.shape[1]] = img1

    return dst

In [ ]:
# Start defining a pipeline
pipeline = dai.Pipeline()

# Define a source - left grayscale cameras
cam_left = pipeline.createMonoCamera()
cam_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

# Create outputs
xout_left = pipeline.createXLinkOut()
xout_left.setStreamName('left')
cam_left.out.link(xout_left.input)

In [ ]:
## RECORDING FOR 5 SECONDS
## THE FRAMES ARE BEING STORED IN DIRECTORY "video_frames"

# Connect and start the pipeline
from pathlib import Path
import time


with dai.Device(pipeline) as device:

    # Output queue will be used to get the grayscale frames from the output defined above
    q = device.getOutputQueue(name="left", maxSize=4, blocking=False)

    # Make sure the destination path is present before starting to store the examples
    Path(f"video_frames/").mkdir(parents=True, exist_ok=True)

    # running loop for 10 secs
    ten_secs = time.time() + 10
    
    while time.time() < ten_secs:
        # Blocking call, will wait until a new data has arrived
        inSrc = q.get()  
        # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
        frame = inSrc.getCvFrame()
        # Frame is transformed and ready to be shown
        cv2.imshow("left", frame)
        cv2.waitKey(1)

        cv2.imwrite(f"video_frames/{int(time.time() * 10000)}.png", frame)

    cv2.destroyAllWindows()            

In [11]:
# Science building stitching using SIFT
img1 = cv2.imread('image_stitching/sciencecenter1.jpg')
img2 = cv2.imread('image_stitching/sciencecenter2.jpg')
img3 = cv2.imread('image_stitching/sciencecenter3.jpg')

In [15]:
# panaroma for science building
img2_3 = stitch_2_imgs(img2, img3, "SIFT")
panorm = stitch_2_imgs(img1, img2_3, "SIFT")

cv2.imwrite(f"image_stitching/sciencecent_stiched.jpg", panorm)

True

In [15]:
# Home building stitching using SIFT

imga = cv2.imread('image_stitching/building3.jpg')
imgb = cv2.imread('image_stitching/building2.jpg')
imgc = cv2.imread('image_stitching/building1.jpg')

In [8]:
# panaroma for home building
imgb_c = stitch_2_imgs(imgb, imgc, "SIFT")
panorm = stitch_2_imgs(imga, imgb_c, "SIFT")

cv2.imwrite(f"image_stitching/building_stiched.jpg", panorm)

True

In [14]:
# Science building stitching using Using ORB

img1_2 = stitch_2_imgs(img1, img2, "ORB")

cv2.imwrite(f"image_stitching/stitch_ORB.png", img1_2)

True